In [ ]:
import os
import cv2

print(cv2.__version__)

4.11.0


In [ ]:
# Get cv2 default haarcascade directory
data_dir = cv2.data.haarcascades

# Define paths to different haarcascade models
cascade_names = {
    "face": "haarcascade_frontalface_default.xml",
    "eyes": "haarcascade_eye.xml"
    # "mouth": "haarcascade_mcs_mouth.xml",
    # "nose": "haarcascade_mcs_nose.xml"
}
cascade_path = {roi: os.path.join(data_dir, filename) for roi, filename in cascade_names.items()}

print(cascade_path)

{'face': '/usr/local/lib/python3.11/dist-packages/cv2/data/haarcascade_frontalface_default.xml', 'eyes': '/usr/local/lib/python3.11/dist-packages/cv2/data/haarcascade_eye.xml'}


In [ ]:
#-- 1. Load the cascades
cascades = {}
for roi, path in cascade_path.items():
    if not os.path.exists(path): # Check if the file exists
        print(f"--(!)Warning: {roi} cascade not found at {path}, Skipping...")
        continue # Skip cascade if file is missing

    classifier = cv2.CascadeClassifier()
    if not classifier.load(path):
        print(f'--(X)Error loading {roi} cascade at {path}')
        exit(0)
    cascades[roi] = classifier
print("--(*)Haar cascades loaded successfully!")

--(*)Haar cascades loaded successfully!


In [ ]:
def detect(frame):
    # Convert the frame into grayscale and equalize
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)

    #-- Detect faces
    faces = cascades['face'].detectMultiScale(
        frame_gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
        )
    # Loop over the bounding box for each face
    for (x,y,w,h) in faces:
        # Obtain the center and draw an ellipse around the face
        center = (x + w//2, y + h//2)
        frame = cv2.ellipse(frame, center, (w//2, h//2), 0, 0, 360, (255, 0, 255), 4)
        face_roi = frame_gray[y:y+h, x:x+w]
        #-- In each face, detect eyes
        eyes = cascades['eyes'].detectMultiScale(
            face_roi,
            scaleFactor=1.1,
            minNeighbors=10,
            minSize=(15, 15),
            flags=cv2.CASCADE_SCALE_IMAGE
            )
        # Loop over the bounding box for the eyes relative to the original frame dimensions
        for (x2, y2, w2, h2) in eyes:
            ptA = (x + x2, y + y2)
            ptB = (x + x2 + w2, y + y2 + h2)
            # Draw rectangles around the eyes
            cv2.rectangle(frame, ptA, ptB, (0, 0, 255), 2)
    cv2.imshow('Capture - Face detection', frame) # Show frame

In [ ]:
#-- 2. Read the video stream
cap = cv2.VideoCapture(1)
if not cap.isOpened:
    print("--(X)Error opening video capture")
    exit(0)

while True:
    ret, frame = cap.read()
    if frame is None:
        print("--(!) No captured frame -- Break!")
        break
    detect(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows() # Clean-up

--(!) No captured frame -- Break!


error: OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
